In [ ]:
!pip install -Uq openai-agents logfire

In [5]:

import nest_asyncio
nest_asyncio.apply()

In [3]:
import asyncio
import os
from agents.run import RunConfig
from openai import AsyncOpenAI
from typing import Optional
from dotenv import load_dotenv
import os

load_dotenv() 
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,OpenAIChatCompletionsModel
)
import logfire
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.0-flash"
logfire_token=os.getenv("LOGFIRE_TOKEN") 
if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )



chat_history: list[dict[str, str]] = []
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)
if logfire_token:
    logfire.configure(token=logfire_token) 
    logfire.instrument_openai(client)

# 3. tell the agents library to use this client by default
set_default_openai_client(client)
@function_tool
def get_weather(city: str, unit: str = None) -> str:
    """
    Returns the weather of a given city.
    Args:
        city: Name of the city (e.g., "Sialkot").
        unit: Optional; temperature unit ("C" or "F").
    """
    unit_info = f" in {unit}" if unit else ""
    return f"The weather in {city}{unit_info} is sunny."
async def main():
     
    agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant that can answer weather  related questions.",
        model=model,
        tools=[get_weather],
    )

    result = await Runner.run(agent, "what is the wether of  Sialkot")
 
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

The weather in Sialkot is sunny.



In [27]:
import asyncio
import os
from agents.run import RunConfig
from openai import AsyncOpenAI
from typing import Optional
from dotenv import load_dotenv
import os

load_dotenv()
from agents import (
    Agent,
    Runner,
    function_tool,
    set_default_openai_api,
    set_default_openai_client,
    set_tracing_disabled,
    OpenAIChatCompletionsModel,
)
import logfire

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.0-flash"
logfire_token = os.getenv("TEST_LOGFIRE_TOKEN")
if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )


chat_history: list[dict[str, str]] = []
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client)

from agents import Tool

@function_tool
def get_weather(city: str, unit: str = None) -> str:
    """
    Returns the weather of a given city.
    Args:
        city: Name of the city (e.g., "Sialkot").
        unit: Optional; temperature unit ("C" or "F").
    """
    unit_info = f" in {unit}" if unit else ""
    return f"  {city}{unit_info} is sunny."


async def main():
    logfire.configure(token=logfire_token)
    logfire.instrument_openai(client)
    agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant that can answer weather  related questions.answer will be user friendly and answer will be one paragraph",
        model=model,
        tools=[get_weather],
        
    )

    result = await Runner.run(agent, "what is the wether of  Sialkot")

    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

04:38:32.823 Chat Completion with 'gemini-2.0-flash' [LLM]


Logfire project URL: ]8;id=104310;https://logfire-us.pydantic.dev/arman/starter-project\https://logfire-us.pydantic.dev/arman/starter-project]8;;\

04:38:34.311 Chat Completion with 'gemini-2.0-flash' [LLM]
The weather in Sialkot is sunny with temperature in Celcius.



In [31]:
import json
from typing_extensions import TypedDict, Any
from agents import Agent, FunctionTool,function_tool
import asyncio
import os 
from openai import AsyncOpenAI 
from dotenv import load_dotenv 
load_dotenv()
from agents import (
    Agent,
    Runner,
    function_tool, 
    OpenAIChatCompletionsModel,
)
from agents import Agent, OpenAIChatCompletionsModel 
import logfire

BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.0-flash"
logfire_token = os.getenv("TEST_LOGFIRE_TOKEN")
if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )


chat_history: list[dict[str, str]] = []
client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client)

from agents import Tool
class Location(TypedDict):
    lat: float
    long: float

@function_tool(strict_mode=True)
async def fetch_weather(location: Location) :
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """ 
    return{"latitude":location["lat"],"longitude":location["long"],"weather":"Sunnay"}
class JSONRunner(Runner):
    async def run(self, agent: Agent, input: str) -> Any:
        # Get the initial response with to 
        response = await super().run(agent, input)
        print(f"response:{response}")
        
agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant that can answer weather  related questions.answer will be user friendly and answer will be one paragraph",
        model=model,
        tools=[fetch_weather],  
        tool_use_behavior="stop_on_first_tool"

        
    )
async def main():
    logfire.configure(token=logfire_token)
    logfire.instrument_openai(client)
    
    runners = JSONRunner()
    result = await runners.run(agent, "what is the wether of latitude is 48 and longitude is 48")
    print(result)


if __name__ == "__main__":
    asyncio.run(main())

 

07:42:36.222 Chat Completion with 'gemini-2.0-flash' [LLM]


Logfire project URL: ]8;id=13369;https://logfire-us.pydantic.dev/arman/starter-project\https://logfire-us.pydantic.dev/arman/starter-project]8;;\

response:RunResult:
- Last agent: Agent(name="Assistant", ...)
- Final output (str):
    {'latitude': 48.0, 'longitude': 48.0, 'weather': 'Sunnay'}
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
None


TypeError: cannot pickle '_contextvars.Context' object

Logfire project URL: https://logfire-us.pydantic.dev/arman/starter-project


In [7]:
import asyncio
import os
from openai import AsyncOpenAI 
from typing_extensions import  Any
from agents import Agent , function_tool, Runner
from agents import OpenAIChatCompletionsModel
import asyncio   
from dotenv import load_dotenv
load_dotenv()
 
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = "gemini-2.0-flash"

if not BASE_URL or not API_KEY or not MODEL_NAME:
    raise ValueError(
        "Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code."
    )

client = AsyncOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=client)


@function_tool(strict_mode=False)
def recommend_product(
    size: str, color: str, discount: str = None, title: str  = None, price: str = None
) -> str:
    """
    Recommends a product based on customer preferences.

    Args:
        size: Preferred product size (e.g., "M", "L").
        color: Preferred product color (e.g., "red", "blue").
        discount: Optional; any discount information (e.g., "20% off").
        title: Optional; product title or category (e.g., "Running Shoes").
        price: Optional; price range or value (e.g., "$50", "under $100").

    Returns:
        A formatted recommendation message.
    """
    message = f"Recommended a product with size={size}, color={color}"
    if discount:
        message += f", discount={discount}"
    if title:
        message += f", title={title}"
    if price:
        message += f", price={price}"
    return message + "."



@function_tool
def track_order(tracking_id: str) -> str:
    """
    Track an order using a tracking ID.

    Args:
        tracking_id (str): The tracking number associated with the order.

    Returns:
        str: The current status of the order.
    """
    return (
        f"Order status for tracking ID '{tracking_id}': In transit, arriving tomorrow."
    )


@function_tool
def get_faq_answer(query: str) -> str:
    """
    Answer frequently asked questions from users.

    Args:
        query (str): A user question or keyword (e.g., "return policy").

    Returns:
        str: The answer to the FAQ query.
    """
    return (
        f"FAQ answer for your query: '{query}' - You can return items within 90 days."
    )


@function_tool
def past_order_history(email: str) -> str:
    """
    Retrieve the past order history for a user.

    Args:
        email (str): The user's email address.

    Returns:
        str: A summary of the user's past orders.
    """
    return f"Past order history for email '{email}': Order #1234, Order #5678, Order #5678."


faq_agent = Agent(
    name="faq_agent",
    instructions="You are a FAQ assistant. Answer the user's question.",
    model=model,
    tools=[
        get_faq_answer,
    ],
)

order_status_agent = Agent(
    name="order_status_agent",
    instructions="You are an order status assistant. Answer the user's question. If any required parameters are missing, ask the user to provide them.",
    model=model,
    tools=[
        track_order,
    ],
)

order_history_agent = Agent(
    name="order_history_agent",
    instructions="You are an order history assistant. Answer the user's question. If any required parameters are missing, ask the user to provide them.",
    model=model,
    tools=[past_order_history],
)
user_recmondation_agent = Agent(
    name="user_recmondation_agent",
    instructions="You are an user recomondation agent size and color are required and other parameters are optional . Answer the user's question. If any required parameters are missing, ask the user to provide them.",
    model=model,
    tools=[recommend_product],
)
 

categorizer = Agent(
    name="categorizer",
    instructions=(
        """You are a categorization assistant for an e-commerce chatbot. Based on the user's input message, your job is to determine the user's intent and respond with **exactly one** of the following categories:

- 'faq' → The user is asking a general question about the store, return policy, shipping, payment methods, hours, etc.
- 'order_status' → The user wants to check the current status of a specific order, including tracking, delivery updates, or delays.
- 'order_history' → The user wants to review past orders. This includes phrases like:
  - "Show me my last orders"
  - "What did I buy recently?"
  - "Show my last 40 orders"
  - "Order history"
- 'recommendation' → The user is asking for product suggestions, best sellers, or what to buy based on their preferences.

⚠️ Important Instructions:
- Only return one of these exact strings: `'faq'`, `'order_status'`, `'order_history'`, or `'recommendation'`.
- Do **not** include any explanation, comments, or additional text.
- Match based on user intent, not keywords alone.
"""
    ),
    model=model,
)


async def main(query: str): 
    intent = await Runner.run(categorizer, query)
    print(f"intent: {intent}")
    label_line = intent.final_output.strip().strip('"').strip("'")
     

    agents = {
        "faq": faq_agent,
        "order_status": order_status_agent,
        "order_history": order_history_agent,
        "recommendation": user_recmondation_agent,
    }
    selected_agent = agents.get(label_line)
    if selected_agent is None:
        raise ValueError(f"Unrecognized intent '{label_line}'. Please make sure the categorizer returns a valid label.")
            
    print(f"current_agent: {selected_agent}")

    result = await Runner.run(selected_agent, query)
    print(result.final_output)
    return result.final_output

if __name__ == "__main__":
    query="hi my email is armanashraf015@gmail.com"
    asyncio.run(main(query))

intent: RunResult:
- Last agent: Agent(name="categorizer", ...)
- Final output (str):
    'faq'
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
current_agent: Agent(name='faq_agent', instructions="You are a FAQ assistant. Answer the user's question.", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x0000019650767CB0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None), tools=[FunctionTool(name='get_faq_answer', description='Answer frequently asked questions from users.', params_json_schema={'properties': {'query': {'description': 'A user question or keyword (e.g., "return policy").', 'title': 'Query', 'ty

In [151]:

if __name__ == "__main__":
    query=" hi i am arman"
    asyncio.run(main(query))

09:17:31.483 Chat Completion with 'gemini-2.0-flash' [LLM]


Logfire project URL: ]8;id=439517;https://logfire-us.pydantic.dev/arman/starter-project\https://logfire-us.pydantic.dev/arman/starter-project]8;;\

intent: RunResult:
- Last agent: Agent(name="categorizer", ...)
- Final output (str):
    faq
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
current_agent: Agent(name='faq_agent', instructions="You are a FAQ assistant. Answer the user's question.", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001FEF5E02870>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None), tools=[FunctionTool(name='get_faq_answer', description='Answer frequently asked questions from users.', params_json_schema={'properties': {'query': {'description': 'A user question or keyword (e.g., "return policy").', 'title': 'Query', 'type

In [14]:
if __name__ == "__main__":
    query = "I want track my order my tracking_id  222   "
    asyncio.run(main(query))

intent: RunResult:
- Last agent: Agent(name="categorizer", ...)
- Final output (str):
    order_status
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
current_agent: Agent(name='order_status_agent', instructions="You are an order status assistant. Answer the user's question. If any required parameters are missing, ask the user to provide them.", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x0000019650767CB0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None), tools=[FunctionTool(name='track_order', description='Track an order using a tracking ID.', params_json_schema={'properties': {'tracking_id': {'d

In [149]:
if __name__ == "__main__":
    query=" what is all my order history my email is ali@gmail.com"
    asyncio.run(main(query))

08:17:44.145 Chat Completion with 'gemini-2.0-flash' [LLM]


Logfire project URL: ]8;id=749773;https://logfire-us.pydantic.dev/arman/starter-project\https://logfire-us.pydantic.dev/arman/starter-project]8;;\

intent: RunResult:
- Last agent: Agent(name="categorizer", ...)
- Final output (str):
    order_history
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
current_agent: Agent(name='order_history_agent', instructions="You are an order history assistant. Answer the user's question. If any required parameters are missing, ask the user to provide them.", handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001FEF56B18E0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None), tools=[FunctionTool(name='past_order_history', description='Retrieve the past order history for a user.', params_json_schema={'properties': {

In [5]:
# ================================== RAG Code Logic ==================================
import os
from dotenv import load_dotenv
load_dotenv()

import fitz
from pinecone import Pinecone, ServerlessSpec
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ====================== Load API Keys ======================
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
PINECONE_API_KEY = os.getenv('FAKE_PINE_CONe')  

# ====================== Initialize Clients ======================
embedding_model = GoogleGenerativeAIEmbeddings(
    google_api_key=GEMINI_API_KEY,
    model="models/embedding-001"
)
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'shoesmartnewnew'

# ====================== Check/Create Index ======================
existing_indexes = [i['name'] for i in pinecone_client.list_indexes()]
if index_name not in existing_indexes:
    pinecone_client.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print(f"Index '{index_name}' created.")

    # ✅ Only store documents if index is newly created
    def extract_text_from_pdf(pdf_path):
        doc = fitz.open(pdf_path)
        return "\n".join(page.get_text("text") for page in doc)

    def store_pdf_to_vector_db(pdf_path):
        pdf_text = extract_text_from_pdf(pdf_path)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            add_start_index=True,
        )
        docs = [Document(page_content=pdf_text)]
        documents = text_splitter.split_documents(docs)
        random_ids = [str(i) for i in range(len(documents))]
        vector_store.add_documents(documents=documents, ids=random_ids)
        print("PDF content added to vector DB.")

    store_pdf_to_vector_db("./combined1.pdf")
else:
    print(f"Index '{index_name}' already exists. Skipping document upload.")

# ====================== Initialize Vector Store ======================
index = pinecone_client.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

# ====================== Search Logic ======================
def search_similar_documents(query, score_threshold=0.65):
    results = vector_store.similarity_search_with_score(query)
    return [res.page_content for res, score in results if score > score_threshold]

# ====================== Run Query ======================
query = "what is return policy"
similar_content = search_similar_documents(query)

# ====================== Print Results ======================
print("Top Matching Chunks:")
for i, chunk in enumerate(similar_content, 1):
    print(f"\n--- Result {i} ---\n{chunk}")


Index 'shoesmartnewnew' already exists. Skipping document upload.
Top Matching Chunks:

--- Result 1 ---
Ensure the product is in original condition with tags intact.

--- Result 2 ---
Ans: You have 30 days from the date of delivery to initiate a return.
Q: How do I initiate a return?
Ans: Log in to your account, navigate to "My Orders," and select "Return Item." Follow the prompts
to complete the process.
Q: Do I need to pay for return shipping?
Ans: Return shipping is free for damaged or defective items. For other returns, a nominal fee may
apply.
Q: When will I receive my refund?
Ans: Refunds are processed within 5-7 business days after we receive the returned item.
Q: Can I return items bought on sale?
Ans: Yes, sale items can be returned unless otherwise specified in the product description.
Q: Can I return items that were a gift?
Ans: Yes, but the refund will be issued as a store credit or to the original payment method.
Q: How do I cancel a return request?
Ans: Go to "My Returns

In [97]:
import os
from dotenv import load_dotenv
import fitz
from pinecone import Pinecone, ServerlessSpec
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ====================== Load Environment Variables ======================
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
PINECONE_API_KEY = os.getenv('FAKE_PINE_CONe')   

# ====================== Initialize Embedding Model ======================
embedding_model = GoogleGenerativeAIEmbeddings(
    google_api_key=GEMINI_API_KEY,
    model="models/embedding-001"
)

# ====================== Pinecone Setup ======================
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'shoesmartnewnew'

existing_indexes = [i['name'] for i in pinecone_client.list_indexes()]
if index_name not in existing_indexes:
    pinecone_client.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.") 
    
index = pinecone_client.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

# ====================== Function to Extract PDF and Store ======================
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    return "\n".join(page.get_text("text") for page in doc)

def store_pdf_to_vector_db(pdf_path):
    pdf_text = extract_text_from_pdf(pdf_path)
    print("Extracted PDF text.",pdf_text)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=200,
        add_start_index=True,
    )
    docs = [Document(page_content=pdf_text)]
    documents = text_splitter.split_documents(docs)
    random_ids = [str(i) for i in range(len(documents))]
    local_db={}
    for i, doc in enumerate(documents):
        local_db[str(i)] = doc.page_content
    print("Local DB created with document IDs.",{k: v for k, v in local_db.items()})   
    print(f"length of local db is {len(local_db)}") 
    vector_store.add_documents(documents=documents, ids=random_ids)
    print("✅ PDF content added to vector DB.")

 
if index_name not in existing_indexes:
    store_pdf_to_vector_db("./combined1.pdf")

 

# ====================== Search Function ======================
def search_similar_documents(query, score_threshold=0.65):
    results = vector_store.similarity_search_with_score(query)
    return [res.page_content for res, score in results]

# ====================== Perform a Query ======================
query = "what is return policy"
similar_content = search_similar_documents(query)

# ====================== Output ======================
print("Top Matching Chunks:")
for i, chunk in enumerate(similar_content, 1):
    print(f"\n--- Result {i} ---\n{chunk}")


Index 'shoesmartnewnew' already exists.
Extracted PDF text. User Manual - ArmanShoeStore
Introduction
Welcome to our Online Shoe Store!
We take pride in offering a wide variety of footwear designed to suit every lifestyle, occasion, and
preference. Whether
you're chasing your fitness goals, walking to the rhythm of your life, or making a style statement, our
curated collection
has something for everyone. Explore categories tailored to your unique needs, from
high-performance athletic shoes to
casual comfort.
Our commitment is to provide you with not only the best products but also an unparalleled shopping
experience. This
guide will walk you through our offerings, answer your questions, and ensure you make the most of
every purchase.
Features by Category
Category Features:
1. Sneakers:
- Style meets comfort for everyday use.
- Available in multiple colors and sizes.
- Ideal for casual and semi-formal occasions.
2. Running:
- Lightweight and durable for optimal performance.
- Designed t